In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110550  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,5.274596e+06
2018-02-28,4.519202e+06
2018-03-31,4.849668e+06
2018-04-30,4.701847e+06
2018-05-31,4.861984e+06
2018-06-30,4.846205e+06
2018-07-31,4.885032e+06
2018-08-31,4.942909e+06
2018-09-30,4.977930e+06


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    5.274596e+06
 2018-02-28    4.519202e+06
 2018-03-31    4.849668e+06
 2018-04-30    4.701847e+06
 2018-05-31    4.861984e+06
 2018-06-30    4.846205e+06
 2018-07-31    4.885032e+06
 2018-08-31    4.942909e+06
 2018-09-30    4.977930e+06
 2018-10-31    4.858744e+06
 2018-11-30    4.690715e+06
 2018-12-31    4.825316e+06
 2019-01-31    4.934653e+06
 2019-02-28    4.650248e+06
 2019-03-31    4.937644e+06
 2019-04-30    4.514491e+06
 2019-05-31    4.675356e+06
 2019-06-30    4.671298e+06
 2019-07-31    4.924873e+06
 2019-08-31    4.767451e+06
 2019-09-30    4.539843e+06
 2019-10-31    4.684235e+06
 2019-11-30    4.952566e+06
 2019-12-31    5.208509e+06
 2020-01-31    5.344736e+06
 2020-02-29    4.619779e+06
 2020-03-31    4.587439e+06
 2020-04-30    4.226836e+06
 2020-05-31    4.574078e+06
 2020-06-30    4.485594e+06
 2020-07-31    5.002455e+06
 2020-08-31    4.651624e+06
 2020-09-30    4.514766e+06
 2020-10-31    4.651316e+06
 2020-11-30    4.735960e+06

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -4.067825
p-value : 0.001094
Critical Values :
	1%: -3.610400
	5%: -2.939109
	10%: -2.608063


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=1146.306, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=1136.957, Time=0.12 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=1137.741, Time=0.06 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=1139.933, Time=0.05 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=1142.957, Time=0.07 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=1142.840, Time=0.08 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=1135.610, Time=0.04 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=1135.687, Time=0.06 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=inf, Time=0.34 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=1140.245, Time=0.13 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.40 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=1136.455, Time=0.08 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=inf, Time=0.29 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=1139.531, Time=0.28 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.48 sec
 ARIMA(3,1,0)(0,0,0)[1] interc

ARIMA(order=(1, 1, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(1,1,0))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   41
Model:                 ARIMA(1, 1, 0)   Log Likelihood                -567.997
Method:                       css-mle   S.D. of innovations         250766.056
Date:                Sat, 11 Sep 2021   AIC                           1141.994
Time:                        01:55:48   BIC                           1147.135
Sample:                             1   HQIC                          1143.866
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3256.6213   2.78e+04     -0.117      0.907   -5.78e+04    5.13e+04
ar.L1.D.y     -0.4208      0.158     -2.661      0.008      -0.731      -0.111
                                    Roots           

# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([4613550.93584233, 4567033.93925653]), array([250766.05602307, 289795.82641927]), array([[4122058.49749197, 5105043.37419269],
       [3999044.55660473, 5135023.32190833]]))
